In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow import keras
import tensorflow as tf

In [2]:
# 리눅스 한글 설정
plt.rc('font', family = 'NanumGothic')

In [3]:
df = pd.read_pickle('mywafer.pkl')

In [5]:
df_train = df[df['train_test_label']=='Training']
df_test = df[df['train_test_label']=='Test']
df_real = df[df['train_test_label'].isna()]

In [6]:
X_tr = df_train['wafermap_reshape'].apply(lambda x: np.array(x))

In [7]:
X_te = df_test['wafermap_reshape'].apply(lambda x: np.array(x))

In [8]:
X_test = np.array([x for x in X_te])

In [9]:
X_train = np.array([x for x in X_tr])

In [10]:
y_train = df_train['failure_type'].astype('category').cat.codes
y_test = df_test['failure_type'].astype('category').cat.codes

### CNN_model 불러오기

In [17]:
m_cnn55 = tf.keras.models.load_model('model_CNN.h5')
m_cnn33 = tf.keras.models.load_model('model_CNN33.h5')

In [15]:
m_cnn55.evaluate(X_test, y_test)

2023-07-16 14:57:27.346994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902


  35/3702 [..............................] - ETA: 16s - loss: 0.3054 - accuracy: 0.9536  

2023-07-16 14:57:29.200586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3702/3702 [==============================] - 21s 4ms/step - loss: 1.0204 - accuracy: 0.7480


[1.0204004049301147, 0.7480180859565735]

In [18]:
m_cnn33.evaluate(X_test, y_test)

3702/3702 [==============================] - 15s 4ms/step - loss: 1.2765 - accuracy: 0.7116


[1.2764838933944702, 0.7115792036056519]

## CNN모델은 kernel size가 (5,5)인것이 (3,3)인것 보다 좋은 정확도를 보인다.
### kernel size가 크면 좋은 성능을 보일까? ==> (10,10)으로 다시 만들어보자

In [19]:
# CNN kernel_size를 (10,10)으로 설정
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(10,10), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (10,10), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 71, 71, 32)        3232      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        102432    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               6

In [20]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10


2023-07-16 15:01:02.447169: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x558a3198a3c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-16 15:01:02.447214: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-07-16 15:01:02.501586: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-16 15:01:02.896749: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1654/1654 [==============================] - 21s 10ms/step - loss: 0.2988 - accuracy: 0.9178
Epoch 2/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.1283 - accuracy: 0.9619
Epoch 3/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0890 - accuracy: 0.9725
Epoch 4/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0698 - accuracy: 0.9787
Epoch 5/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0541 - accuracy: 0.9830
Epoch 6/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0435 - accuracy: 0.9862
Epoch 7/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0364 - accuracy: 0.9888
Epoch 8/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0293 - accuracy: 0.9905
Epoch 9/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0222 - accuracy: 0.9927
Epoch 10/10
1654/1654 [==============================] - 16s 10ms/step - lo

In [23]:
model.save('model_CNN10.h5')

In [29]:
m_cnn10 = tf.keras.models.load_model('model_CNN10.h5')

In [30]:
m_cnn10.evaluate(X_test, y_test)

3702/3702 [==============================] - 17s 4ms/step - loss: 1.3408 - accuracy: 0.7200


[1.340801477432251, 0.7199544310569763]

### 10,10으로 했을 때도 5,5보다 좋지 못한 성능을 보인다. kernel size를 10과 5사이에서 조율이 필요하다.

In [31]:
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(8,8), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (8,8), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.001),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 73, 73, 32)        2080      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        65568     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [32]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1654/1654 [==============================] - 19s 9ms/step - loss: 0.2756 - accuracy: 0.9242
Epoch 2/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.1173 - accuracy: 0.9643
Epoch 3/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.0836 - accuracy: 0.9738
Epoch 4/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0675 - accuracy: 0.9790
Epoch 5/10
1654/1654 [==============================] - 16s 9ms/step - loss: 0.0523 - accuracy: 0.9831
Epoch 6/10
1654/1654 [==============================] - 16s 10ms/step - loss: 0.0400 - accuracy: 0.9873
Epoch 7/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0312 - accuracy: 0.9902
Epoch 8/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0266 - accuracy: 0.9916
Epoch 9/10
1654/1654 [==============================] - 15s 9ms/step - loss: 0.0226 - accuracy: 0.9927
Epoch 10/10
1654/1654 [==============================] - 15s 9ms/step -

In [33]:
model.save('model_CNN8.h5')

In [34]:
m_cnn8 = tf.keras.models.load_model('model_CNN8.h5')

In [35]:
m_cnn8.evaluate(X_test, y_test)

3702/3702 [==============================] - 16s 4ms/step - loss: 1.0979 - accuracy: 0.7371


[1.097948670387268, 0.7370594143867493]

### kernel size는 5,5가 가장 높은 정확도를 보여준다. 이제 5,5모델의 learning_rate를 바꿔보자

In [36]:
#learning_rate = 0.01

model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(5,5), activation = 'relu', input_shape=(80, 80, 1),
                       padding = 'valid', data_format = 'channels_last'), # 2D는 평면
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Conv2D(32, kernel_size = (5,5), activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(9)
])


model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         optimizer = keras.optimizers.Adam(learning_rate = 0.01),
         metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 76, 76, 32)        832       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 32)        25632     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 9248)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [37]:
model.fit(X_train, y_train, epochs = 10)

Epoch 1/10
1654/1654 [==============================] - 17s 9ms/step - loss: 0.3598 - accuracy: 0.9050
Epoch 2/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2724 - accuracy: 0.9274
Epoch 3/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2500 - accuracy: 0.9326
Epoch 4/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2352 - accuracy: 0.9357
Epoch 5/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2245 - accuracy: 0.9387
Epoch 6/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2166 - accuracy: 0.9418
Epoch 7/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2099 - accuracy: 0.9420
Epoch 8/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.2022 - accuracy: 0.9451
Epoch 9/10
1654/1654 [==============================] - 14s 9ms/step - loss: 0.1892 - accuracy: 0.9480
Epoch 10/10
1654/1654 [==============================] - 14s 9ms/step - l

In [38]:
model.save('m_cnn5_rate001.h5')

In [44]:
m_cnn5_rate001 = tf.keras.models.load_model('m_cnn5_rate001.h5')

In [45]:
m_cnn5_rate001.evaluate(X_test, y_test)

2023-07-16 15:21:15.797633: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 722.93MiB (rounded to 758048000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-16 15:21:15.797706: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-07-16 15:21:15.797723: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 218, Chunks in use: 218. 54.5KiB allocated for chunks. 54.5KiB in use in bin. 10.0KiB client-requested in use in bin.
2023-07-16 15:21:15.797735: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 33, Chunks in use: 33. 17.2KiB allocated for chunks. 17.2KiB in use in bin. 16.5KiB client-requested in use in bin.
2023-07-16 15:21:15.797746: I tensorflow/tsl/f

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

ks in use: 3. 1.73GiB allocated for chunks. 1.34GiB in use in bin. 1.34GiB client-requested in use in bin.
2023-07-16 15:21:15.797958: I tensorflow/tsl/framework/bfc_allocator.cc:1062] Bin for 722.93MiB was 256.00MiB, Chunk State: 
2023-07-16 15:21:15.797979: I tensorflow/tsl/framework/bfc_allocator.cc:1068]   Size: 405.02MiB | Requested Size: 0B | in_use: 0 | bin_num: 20, prev:   Size: 322.91MiB | Requested Size: 322.91MiB | in_use: 1 | bin_num: -1, next:   Size: 4.52MiB | Requested Size: 4.52MiB | in_use: 1 | bin_num: -1
2023-07-16 15:21:15.797988: I tensorflow/tsl/framework/bfc_allocator.cc:1075] Next region of size 2176450560
2023-07-16 15:21:15.798000: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc2000000 of size 256 next 1
2023-07-16 15:21:15.798008: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc2000100 of size 1280 next 2
2023-07-16 15:21:15.798016: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc2000600 of size 256 next 3
2023

2023-07-16 15:21:15.798791: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c4600 of size 256 next 151
2023-07-16 15:21:15.798799: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c4700 of size 256 next 141
2023-07-16 15:21:15.798806: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c4800 of size 256 next 142
2023-07-16 15:21:15.798814: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c4900 of size 2304 next 87
2023-07-16 15:21:15.798821: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c5200 of size 256 next 98
2023-07-16 15:21:15.798829: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c5300 of size 512 next 104
2023-07-16 15:21:15.798836: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c5500 of size 256 next 103
2023-07-16 15:21:15.798844: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7fdfc29c5600 of size 4096 next 90
2023-07-16 15:21:15.798852: I ten